In [1]:
#!/usr/bin/env python
# coding: utf-8

import os, shutil, sys
from datetime import datetime
#import AnaUtils as mc
import configparser
import argparse
import pennylane as qml
import numpy as np
from pennylane.templates import QAOAEmbedding, StronglyEntanglingLayers
from pennylane.optimize import AdagradOptimizer,AdamOptimizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import time
import pandas as pd
from os.path import basename
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn import metrics

In [2]:
df_train = pd.read_csv('/mldata/bb_asym/MC/csv/v2/trainData.csv')
df_test = pd.read_csv('/mldata/bb_asym/MC/csv/v2/testData.csv')
inputpath = "/mldata/bb_asym/MC/hdf/newv2/"

In [3]:
fw = { "bb": [1.,0.2497949295,0.1465910663,0.0045055782],       "cc": [1.,0.2078621513,0.1203924034,0.0034683555],       "qq": [1, 0.2164989008, 0.1153463426, 0.0034510991]}

fnames = ["Dijet_bb_pt10_15_dw.hdf", "Dijet_bb_pt10_15_up.hdf", "Dijet_bb_pt15_20_dw.hdf", "Dijet_bb_pt15_20_up.hdf","Dijet_bb_pt20_50_dw.hdf", "Dijet_bb_pt20_50_up.hdf","Dijet_bb_pt50_dw.hdf", "Dijet_bb_pt50_up.hdf"]
#fnames = ["Dijet_bb_pt10_15_dw.hdf", "Dijet_bb_pt10_15_up.hdf", "Dijet_bb_pt15_20_dw.hdf", "Dijet_bb_pt15_20_up.hdf"]

In [4]:
columns_to_import = ['idx', 'Jet_foundMuon', 'Jet_muon_q', 'Jet_muon_PT', 
                       'Jet_LABEL', 'Jet_PT', 'Jet_ETA', 'Jet_PHI']

In [5]:
def getFileWeight(wtable, fname):
    for k in wtable.keys():
        if k in fname:
            key=k
    for i,j in enumerate(["pt10", "pt15", "pt20", "pt50"]):
        if j in fname:
            index=i
    return wtable[key][index]

In [6]:
indata = pd.DataFrame()
weights=[]
for fn in fnames:
    temp = pd.read_hdf(inputpath+fn, columns=columns_to_import)
    print(inputpath+fn)
    indata = pd.concat([indata, temp], axis=0)
    weights = np.append(weights, np.full((1, temp.shape[0]), getFileWeight(fw, fn)))
indata['weights'] = weights.tolist()

# use 'idx' as index
indata.set_index('idx', inplace=True)
idxjet = df_test['idx']


/mldata/bb_asym/MC/hdf/newv2/Dijet_bb_pt10_15_dw.hdf
/mldata/bb_asym/MC/hdf/newv2/Dijet_bb_pt10_15_up.hdf
/mldata/bb_asym/MC/hdf/newv2/Dijet_bb_pt15_20_dw.hdf
/mldata/bb_asym/MC/hdf/newv2/Dijet_bb_pt15_20_up.hdf
/mldata/bb_asym/MC/hdf/newv2/Dijet_bb_pt20_50_dw.hdf
/mldata/bb_asym/MC/hdf/newv2/Dijet_bb_pt20_50_up.hdf
/mldata/bb_asym/MC/hdf/newv2/Dijet_bb_pt50_dw.hdf
/mldata/bb_asym/MC/hdf/newv2/Dijet_bb_pt50_up.hdf


In [7]:
# filter input data 
jet_df = indata.loc[idxjet]

In [8]:
prediction = pd.read_csv('/lhcbdata/users/zuliani/lhcb_mljet/BBAsym/results/article/results/DN_TestProb.csv')
prediction.set_index('idx', inplace=True)
print(prediction)
prob = prediction['prob']
jet_df['prob'] = prob

                     prob
idx                      
67534_bbdw50V2   0.401224
206375_bbup50V2  0.359896
210480_bbup20V2  0.642130
152795_bbdw20V2  0.416559
70915_bbup50V2   0.409409
...                   ...
29075_bbup50V2   0.484026
1599_bbdw50V2    0.427681
201530_bbdw20V2  0.589480
11186_bbdw15V2   0.394622
124949_bbup50V2  0.431559

[295218 rows x 1 columns]


In [9]:
print( jet_df)

                 Jet_foundMuon  Jet_muon_q  Jet_muon_PT  Jet_LABEL  \
idx                                                                  
67534_bbdw50V2           False           0        -99.0         -1   
206375_bbup50V2          False           0        -99.0          1   
210480_bbup20V2          False           0        -99.0         -1   
152795_bbdw20V2          False           0        -99.0         -1   
70915_bbup50V2           False           0        -99.0         -1   
...                        ...         ...          ...        ...   
29075_bbup50V2           False           0        -99.0          1   
1599_bbdw50V2            False           0        -99.0         -1   
201530_bbdw20V2          False           0        -99.0          1   
11186_bbdw15V2           False           0        -99.0         -1   
124949_bbup50V2          False           0        -99.0         -1   

                       Jet_PT   Jet_ETA   Jet_PHI   weights      prob  
idx              

In [10]:
def MC_ApplyAlgTagging(df, netype="MuTag", ProbLcut=0.5, ProbRcut=0.5, mu_cut=5000):
    """
    Performs the MLAlg tagging for each jet, and applies a probabilities R/L cuts or a muonPT cut.
    netype is used to identify columns name. 
    -> Right cut is for label 1, left cut for label 0 

    COLUMNS REQUIRED          COLUMNS ADDED
    Jet_foundMuon (if MuTag)  "Jet_%s_Charge"%nettype (0: Not tagged)    
    Jet_muon_q    (if MuTag)
    Jet_muon_PT   (if MuTag)
    Jet_%sProb"%netype       
    """
    if (netype=="MuTag"):
      df['Jet_MuTag_Charge'] = 0
      tt_Jet = (df['Jet_foundMuon'] == 1) & (df['Jet_muon_PT'] > mu_cut)
      df.loc[tt_Jet, 'Jet_MuTag_Charge'] = df.loc[tt_Jet].Jet_muon_q
    else:
      df['Jet_%s_Charge'%netype] = 0
      tt_Jet_1 = (df["Jet_%sProb"%netype] > ProbRcut)
      tt_Jet_0 = (df["Jet_%sProb"%netype] < ProbLcut)
      df.loc[tt_Jet_1,'Jet_%s_Charge'%netype] =  1
      df.loc[tt_Jet_0,'Jet_%s_Charge'%netype] =  -1

def MC_ApplyAlgChecking(df, netype="MuTag"):
    """
    Checks the result of the Alg tagging by comparing the guess with the MC-truth.
    netype is used to identify columns name. 

    COLUMNS REQUIRED           COLUMNS ADDED

    "Jet_%s_Charge"%netype     "Jet_%s_Check"%netype     
    Jet_MATCHED_CHARGE
    """
    df['Jet_%s_Check'%netype] = 0

    tt_correct_jet = (df['Jet_%s_Charge'%netype] != 0) & (df['Jet_%s_Charge'%netype] == df.Jet_MATCHED_CHARGE)
    tt_notcorrect_jet = (df['Jet_%s_Charge'%netype] != 0) & (df['Jet_%s_Charge'%netype] != df.Jet_MATCHED_CHARGE)

    df.loc[tt_correct_jet, 'Jet_%s_Check'%netype] = 1
    df.loc[tt_notcorrect_jet, 'Jet_%s_Check'%netype] = -1

In [11]:
jet_df.rename(columns={"Jet_LABEL": "Jet_MATCHED_CHARGE"}, inplace=True)
jet_df.rename(columns={"prob": "Jet_QuantumProb"}, inplace=True)
print(jet_df)
MC_ApplyAlgTagging(jet_df, "Quantum")
MC_ApplyAlgChecking(jet_df, "Quantum")

                 Jet_foundMuon  Jet_muon_q  Jet_muon_PT  Jet_MATCHED_CHARGE  \
idx                                                                           
67534_bbdw50V2           False           0        -99.0                  -1   
206375_bbup50V2          False           0        -99.0                   1   
210480_bbup20V2          False           0        -99.0                  -1   
152795_bbdw20V2          False           0        -99.0                  -1   
70915_bbup50V2           False           0        -99.0                  -1   
...                        ...         ...          ...                 ...   
29075_bbup50V2           False           0        -99.0                   1   
1599_bbdw50V2            False           0        -99.0                  -1   
201530_bbdw20V2          False           0        -99.0                   1   
11186_bbdw15V2           False           0        -99.0                  -1   
124949_bbup50V2          False           0        -9

In [12]:
print(jet_df)

                 Jet_foundMuon  Jet_muon_q  Jet_muon_PT  Jet_MATCHED_CHARGE  \
idx                                                                           
67534_bbdw50V2           False           0        -99.0                  -1   
206375_bbup50V2          False           0        -99.0                   1   
210480_bbup20V2          False           0        -99.0                  -1   
152795_bbdw20V2          False           0        -99.0                  -1   
70915_bbup50V2           False           0        -99.0                  -1   
...                        ...         ...          ...                 ...   
29075_bbup50V2           False           0        -99.0                   1   
1599_bbdw50V2            False           0        -99.0                  -1   
201530_bbdw20V2          False           0        -99.0                   1   
11186_bbdw15V2           False           0        -99.0                  -1   
124949_bbup50V2          False           0        -9

In [13]:
def MC_GetAlgPerformances(jet_df, netype="MuTag"):
    """
    Extract Alg performance and associated uncertainties.
    netype is used to identify columns name.

    COLUMNS REQUIRED
    'Jet_%s_Check'%netype
    """
    N_tot = len(jet_df)
    print(N_tot)

    #jet_df.drop(jet_df[(jet_df.Jet_muon_q == 0)].index,inplace=True)      #considera solo i muoni

    N_tag = jet_df['Jet_%s_Check'%netype].value_counts()[1] + jet_df['Jet_%s_Check'%netype].value_counts()[-1]
    N_wrong = jet_df['Jet_%s_Check'%netype].value_counts()[-1]
    #Efficiency
    eps_eff = float(N_tag)/N_tot
    err_eps_eff = np.sqrt(eps_eff*(1-eps_eff)/N_tot)
    #Omega
    omega = float(N_wrong)/(N_tag)
    err_omega  = np.sqrt(omega*(1-omega)/N_tag)

    #Accuracy
    accuracy = 1- omega
    
    #Tagging Power
    eps_tag = (eps_eff * (1-2*omega)**2)
    #err_eps_tag = np.sqrt((N_tot*(eps_eff)*(1-eps_eff)) * Deps_DNtag(N_tot,N_wrong,N_tag)**2 + (N_tag*(omega)*(1-omega)) * Deps_DNw(N_tot,N_wrong,N_tag)**2)
    err_eps_tag = 0
    #print('tagging power = %s'%eps_tag)
    
    return eps_eff,omega,eps_tag,err_eps_eff,err_omega,err_eps_tag,N_tot,accuracy

In [27]:
def MCAnalysis(jet_df, netype="Quantum", nbins=8, basepath="", legend=True):
    '''
    Performs the algorithms evaluation
    '''

    # Discretize variable into equal-sized buckets using jet PT
    out,bins = pd.qcut(jet_df.Jet_PT,nbins,retbins=True,labels=False)
    bin_centers = []
    half_widths = []

    omegas = []
    err_omegas = []
    effs = []
    err_effs = []
    ptags = []
    err_ptags = []
    tot = []
    accuracies = []

    for i in range(nbins):
        bin_center = float((bins[i] + bins[i+1])/2)/1000
        half_width = float((bins[i+1] - bins[i])/2)/1000
        bin_centers.append(bin_center)
        half_widths.append(half_width)
    
        eps_eff,omega,eps_tag,err_eps_eff,err_omega,err_eps_tag,N_tot,accuracy = MC_GetAlgPerformances(jet_df[out == i], netype)
        omegas.append(omega*100)
        err_omegas.append(err_omega*100)
        ptags.append(eps_tag*100)
        err_ptags.append(err_eps_tag*100)
        effs.append(eps_eff*100)
        err_effs.append(err_eps_eff*100)

    print(bin_centers)
    print(ptags)
    print(half_widths)
    print(err_ptags)
 
    fig = plt.figure(figsize=(6,6))
    plt.errorbar(bin_centers, ptags, xerr=half_widths, yerr=err_ptags, fmt='o', capsize=2, ms=8)
  
    #plt.xticks(np.arange(20,101, step=80))
    #plt.yticks(np.arange(1,40, step=4))
    plt.xlabel("$\mathrm{P_{T} \ (GeV)}$")
    plt.ylabel("$\mathrm{\epsilon_{tag} \ (\%)}$")

    plt.minorticks_on()
    plt.figtext(0.530,0.360,"LHCb Simulation\nOpen Data", ma='right', fontsize=18)

    fig.savefig("./TagPower.png", bbox_inches='tight')
    plt.close()

    fig = plt.figure(figsize=(6,6))
    plt.errorbar(bin_centers, effs, xerr=half_widths, yerr=err_effs, fmt='o', capsize=2, ms=8)
  
    #plt.xticks(np.arange(20,101, step=80))
    #plt.yticks(np.arange(1,40, step=4))
    plt.xlabel("$\mathrm{P_{T} \ (GeV)}$")
    plt.ylabel("$\mathrm{\epsilon_{tag} \ (\%)}$")

    plt.minorticks_on()
    plt.figtext(0.530,0.360,"LHCb Simulation\nOpen Data", ma='right', fontsize=18)

    fig.savefig("./efficiency.png", bbox_inches='tight')
    plt.close()


    fig = plt.figure(figsize=(6,6))
    plt.errorbar(bin_centers, omegas, xerr=half_widths, yerr=err_omegas, fmt='o', capsize=2, ms=8)
  
    #plt.xticks(np.arange(20,101, step=80))
    #plt.yticks(np.arange(1,40, step=4))
    plt.xlabel("$\mathrm{P_{T} \ (GeV)}$")
    plt.ylabel("$\mathrm{\epsilon_{tag} \ (\%)}$")

    plt.minorticks_on()
    plt.figtext(0.530,0.360,"LHCb Simulation\nOpen Data", ma='right', fontsize=18)

    fig.savefig("./mistag.png", bbox_inches='tight')
    plt.close()

In [28]:
MCAnalysis(jet_df)

36903
36902
36902
36902
36902
36902
36902
36903
[21.704901243007757, 25.52512511551025, 30.589308717134635, 36.82852041315262, 43.046542394278504, 48.98353372151615, 56.042582469236976, 80.04332874453804]
[7.971292208608471, 7.845221182256748, 8.157957016262054, 7.872569661817728, 7.0612887137584845, 6.88952246159075, 6.957975172797346, 6.606194012374993]
[1.7047500412834489, 2.1154738312190475, 2.948709770405336, 3.290501925612647, 2.9275200555132397, 3.009471271724411, 4.049577475996411, 19.951168799304657]
[0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
fig = plt.figure(figsize=(6,6))
plt.errorbar(bin_centers, omegas, xerr=half_widths, yerr=err_omegas, fmt='o', capsize=2, ms=8)
  
plt.xlabel("prob")
plt.ylabel("events")

plt.hist()

plt.minorticks_on()
plt.figtext(0.530,0.360,"LHCb Simulation\nOpen Data", ma='right', fontsize=18)

fig.savefig("./prob.png", bbox_inches='tight')
plt.close()